# Ex. 12

## a)

In [2]:
import tensorflow as tf

In [3]:
# def build(self, input_shape):
#     self.alpha = self.add_weight(name="alpha", shape=[input_shape[-1:]], dtype=tf.float32, initializer="ones")
#     self.beta = self.add_weight(name="beta", shape=[input_shape[-1:]], dtype=tf.float32, initializer="zeros")

## b)

In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, eps, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps

    def build(self, input_shape):
        self.alpha = self.add_weight(name="alpha", shape=input_shape[-1:], dtype=tf.float32, initializer="ones")
        self.beta = self.add_weight(name="beta", shape=input_shape[-1:], dtype=tf.float32, initializer="zeros")

    def call(self, X):
        mu, var = tf.nn.moments(X, axes=-1, keepdims=True)
        std = tf.sqrt(var)
        return self.alpha * (X - mu)/(std + self.eps) + self.beta

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.eps}